In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

image_transforms = transforms.Compose(
         [transforms.ToTensor(),
         transforms.CenterCrop(28),
         transforms.Normalize((0.5), (0.5))]
)
batchsize = 25
rowsize = 28
colsize = 28
inchannels = 3
kernelsize = 5
stride = 1
padding = 0
poolsize = 3
fc_size1 = 384
fc_size2 = 192
no_classes = 10
epochs = 100

train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms)
test = torchvision.datasets.CIFAR10(root='./', train=False, download=True, transform=image_transforms)

evens = list(range(0, len(train), 100))

odds = list(range(1, len(test), 5))
trainset_1 = torch.utils.data.Subset(train, evens)
trainset_2 = torch.utils.data.Subset(train, odds)



traindata = torch.utils.data.DataLoader(trainset_1,batch_size=batchsize, shuffle=True, num_workers=2,pin_memory=True)

testdata = torch.utils.data.DataLoader(trainset_2,batch_size=batchsize, shuffle=False, num_workers=2,pin_memory=True)

class SimpleAlex(nn.Module):
  def __init__(self,inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes):
    super(SimpleAlex, self).__init__()
    self.conv1 = nn.Conv2d(inchannels,24,kernelsize,stride,padding)
    self.pool1 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm1 = nn.BatchNorm2d(24)
    self.conv2 = nn.Conv2d(24,48,kernelsize,stride,padding)
    self.pool2 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm2 = nn.BatchNorm2d(48)
    self.fc1 = nn.Linear(432,fc_size1)
    self.fc2 = nn.Linear(fc_size1,fc_size2)
    self.fc3 = nn.Linear(fc_size2,no_classes) 

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = self.BatchNorm1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = self.BatchNorm2(x)
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return x

model = SimpleAlex(inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes).to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay = 0.005, momentum = 0.9)
criterion = nn.CrossEntropyLoss().to(device)
list_data = []
for i,data in enumerate(traindata):
    list_data.append(data[1])

k = len(list_data)
   

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  
  for i,data in enumerate(traindata):
    if(i <= k/5):
      data[1] = random.sample(list_data,1)[0]

    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    # pred[i] = outputs.argmax(dim=1)
    # act[i] = labels.argmax(dim=1)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
  #print(num_correct/tot)
  print(Loss)
  
for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
print(num_correct/tot)

Device available now: cuda


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified
tensor(45.5182, device='cuda:0', grad_fn=<AddBackward0>)
tensor(43.8824, device='cuda:0', grad_fn=<AddBackward0>)
tensor(42.5508, device='cuda:0', grad_fn=<AddBackward0>)
tensor(42.3892, device='cuda:0', grad_fn=<AddBackward0>)
tensor(40.0811, device='cuda:0', grad_fn=<AddBackward0>)
tensor(39.9481, device='cuda:0', grad_fn=<AddBackward0>)
tensor(39.4754, device='cuda:0', grad_fn=<AddBackward0>)
tensor(38.4530, device='cuda:0', grad_fn=<AddBackward0>)
tensor(37.5246, device='cuda:0', grad_fn=<AddBackward0>)
tensor(35.1367, device='cuda:0', grad_fn=<AddBackward0>)
tensor(36.0493, device='cuda:0', grad_fn=<AddBackward0>)
tensor(35.7027, device='cuda:0', grad_fn=<AddBackward0>)
tensor(36.2748, device='cuda:0', grad_fn=<AddBackward0>)
tensor(32.8493, device='cuda:0', grad_fn=<AddBackward0>)
tensor(33.6749, device='cuda:0', grad_fn=<AddBackward0>)
tensor(33.6179, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3

In [2]:
list_data = []
for i,data in enumerate(traindata):
    list_data.append(data[1])

k = len(list_data)
   

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  
  for i,data in enumerate(traindata):
    if(i <= 2*k/5):
      data[1] = random.sample(list_data,1)[0]

    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    # pred[i] = outputs.argmax(dim=1)
    # act[i] = labels.argmax(dim=1)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
  #print(num_correct/tot)
  #print(Loss)
  
for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
print(num_correct/tot)

0.3964


In [4]:
list_data = []
for i,data in enumerate(traindata):
    list_data.append(data[1])

k = len(list_data)
   

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  
  for i,data in enumerate(traindata):
    if(i <= 3*k/5):
      data[1] = random.sample(list_data,1)[0]

    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    # pred[i] = outputs.argmax(dim=1)
    # act[i] = labels.argmax(dim=1)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
  #print(num_correct/tot)
  #print(Loss)
  
for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
print(num_correct/tot)

0.3628


In [6]:
list_data = []
for i,data in enumerate(traindata):
    list_data.append(data[1])

k = len(list_data)
   

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  
  for i,data in enumerate(traindata):
    if(i <= 4*k/5):
      data[1] = random.sample(list_data,1)[0]

    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    # pred[i] = outputs.argmax(dim=1)
    # act[i] = labels.argmax(dim=1)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
  #print(num_correct/tot)
  #print(Loss)
  
for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
print(num_correct/tot)

0.2552


In [8]:
list_data = []
for i,data in enumerate(traindata):
    list_data.append(data[1])

k = len(list_data)
   

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  
  for i,data in enumerate(traindata):
    if(i <= k):
      data[1] = random.sample(list_data,1)[0]

    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    # pred[i] = outputs.argmax(dim=1)
    # act[i] = labels.argmax(dim=1)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
  #print(num_correct/tot)
  #print(Loss)
  
for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
print(num_correct/tot)

0.0864
